In [2]:
import openai
import re
import string
import numpy as np
from tqdm import tqdm
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

In [28]:


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

def call_instructgpt_with_answers(questions, model_predictions, ground_truths):
    # if exact_match_score(model_prediction, ground_truth):
    #     return 1
    api_key = open("/data/katie_kang/openai_key_file.txt", "r").read()
    openai.api_key = api_key.strip()

    prompt_template1 = """Are the answers equivalent?

Ex 1:
Q: Who was leader of Revolutionary Army?
A1: George Washington
A2: US President Washington

Ex 2:
Q: Who is the best RL Researcher?
A1: Professor Sergey Levine
A2: Professor Pieter Abbeel

Ex 3:
Q: country of citizenship of McKean?
A1: United States
A2: American

Ex 4:
Q: What genre is Taxi for Two?
A1: romantic comedy film
A2: comedy

"""


    prompt_template2 = None

    for i in range(len(questions)):
        if i == 0:
            prompt_template2 = """Ex {}:
Q: {}
A1: {}
A2: {}

""".format(i+5, questions[i], model_predictions[i], ground_truths[i])
        else:
            prompt_template2 += """Ex {}:
Q: {}
A1: {}
A2: {}

""".format(i+5, questions[i], model_predictions[i], ground_truths[i])

    prompt_template3 = """Ex 1 Equivalent? Yes
Ex 2 Equivalent? No
Ex 3 Equivalent? Yes
Ex 4 Equivalent? Yes
"""

    prompt = prompt_template1 + prompt_template2 + prompt_template3

    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",  # or another model version
        # prompt=[filled_prompt, filled_prompt],
        prompt=prompt,
        max_tokens=50,
        temperature=0.0,
    )
    print(prompt)
    model_response = response.choices[0].text.strip()
    print(model_response)
    1/0
    exit()


    # Interpret the model's response
    if model_response == "Yes":
        return 1
    elif model_response == "No":
        return 0
    else:
        print('WARNING', model_response)
        return 0


    

In [29]:

import numpy as np

data = np.load('ckpts/sft_ctrex_llama7B_2_commit_lr1e-5_2/checkpoint_30000/hf_model/ood_output_strings.npy')

equivalent_all = []

num_points_per_prompt = 4
for i in tqdm(range(len(data)//num_points_per_prompt)):


    # line = data[i]
    # # try:
    # answer = line.split('Label: ')[1].strip()
    # prediction = line.split('The answer is')[1].split('Label:')[0].strip()[0:-1]
    # question = line.split('<unk>')[-1].split('?')[0].strip() + '?'

    answers = []
    predictions = []
    questions = []

    for j in range(i, min(i+num_points_per_prompt, len(data))):
        line = data[j]
        answer = line.split('Label: ')[1].strip()
        prediction = line.split('The answer is')[1].split('Label:')[0].strip()[0:-1]
        question = line.split('<unk>')[-1].split('?')[0].strip() + '?'

        answers.append(answer)
        predictions.append(prediction)
        questions.append(question)


    result = int(call_instructgpt_with_answers(questions, answers, predictions))
    print(question, answer, prediction)
    print(result)
    # except:
    #     print('WARNING', line)
    #     result = 0
    equivalent_all.append(result)

  0%|          | 0/82374 [00:00<?, ?it/s]

  0%|          | 0/82374 [00:00<?, ?it/s]

Are the answers equivalent?

Ex 1:
Q: Who was leader of Revolutionary Army?
A1: George Washington
A2: US President Washington

Ex 2:
Q: Who is the best RL Researcher?
A1: Professor Sergey Levine
A2: Professor Pieter Abbeel

Ex 3:
Q: country of citizenship of McKean?
A1: United States
A2: American

Ex 4:
Q: What genre is Taxi for Two?
A1: romantic comedy film
A2: comedy

Ex 5:
Q: Who approved the Apple Public Source License?
A1: OSI
A2: Apple

Ex 6:
Q: Who approved the GPL?
A1: OSI
A2: Linus Torvalds

Ex 7:
Q: Who approved the zlib license?
A1: FSF
A2: GNU Project

Ex 8:
Q: Who approved the MIT?
A1: Free Software Foundation
A2: MIT

Ex 1 Equivalent? Yes
Ex 2 Equivalent? No
Ex 3 Equivalent? Yes
Ex 4 Equivalent? Yes

Ex 5 Equivalent? No
Ex 6 Equivalent? No
Ex 7 Equivalent? No
Ex 8 Equivalent? No


ZeroDivisionError: division by zero

In [1]:
equivalent_all

NameError: name 'equivalent_all' is not defined

In [ ]:
    
    # prompt_template = """
    # Are the answers equivalent? Answer "Yes" or "No"

    # Ex 1:
    # Q: Who was leader of Revolutionary Army?
    # A1: George Washington
    # A2: US President Washington
    # Equivalent? Yes

    # Ex 2:
    # Q: Who is the best RL Researcher?
    # A1: Professor Sergey Levine
    # A2: Professor Pieter Abbeel
    # Equivalent? No

    # Ex 3:
    # Q: country of citizenship of McKean?
    # A1: United States
    # A2: American
    # Equivalent? Yes

    # Ex 4:
    # Q: What genre is Taxi for Two?
    # A1: romantic comedy film
    # A2: comedy
    # Equivalent? Yes

    # Ex 5:
    # Q: {}
    # A1: {}
    # A2: {}
    # Equivalent?"""
